In [3]:
from PIL import Image, ImageDraw
import numpy as np

def Gaussian_noise(img, amp):
    pixel = img.load()
    Gaussian_noise_img = Image.new(img.mode, img.size)
    
    for i in range(0,512,1):
        for j in range(0,512,1):
            Gaussian_noputpixelise_img.((i,j),int( pixel[i,j] + amp * np.random.normal(0,1)) ) 
            
    return Gaussian_noise_img

def salt_and_pepper_noise(img, threshold):
    pixel = img.load()
    salt_and_pepper_noise_img = Image.new(img.mode, img.size)
    
    for i in range(0,512,1):
        for j in range(0,512,1):
            rand = np.random.sample()
            if rand < threshold:
                salt_and_pepper_noise_img.putpixel((i,j),0)
            elif rand > 1 - threshold:
                salt_and_pepper_noise_img.putpixel((i,j),255)
            else:
                salt_and_pepper_noise_img.putpixel((i,j),pixel[i,j])
    return salt_and_pepper_noise_img

def box_and_median_filter(img, box_size):
    pixel = img.load()
    img_box = Image.new(img.mode, img.size)
    img_median = Image.new(img.mode, img.size)
    
    x_start = int(box_size/2)
    y_start = int(box_size/2)
    for i in range(0,512,1):
        for j in range(0,512,1):
            box_collect = []
            for x in range(0,box_size,1):
                for y in range(0,box_size,1):
                    try:
                        box_collect.append( pixel[i+x-x_start, j+y-y_start] )
                    except:
                        pass
            img_box.putpixel((i,j) , int(np.mean(np.array(box_collect))) )
            img_median.putpixel((i,j) ,int( np.median(np.array(box_collect))) )

    return img_box, img_median

def dilation(img, kernel):
    pixel = img.load()
    coulmn,row=img.size
    img_new = Image.new(img.mode, img.size)

    for i in range(0,coulmn,1):
        for j in range(0,row,1):
            if pixel[i,j] > 0:
                dil_pix_list = []
                for y in range(-2,3,1):
                    for x in range(-2,3,1):
                        if kernel[y+2,x+2] == 1:
                            if (i+x < coulmn) and (j+y < row) and (i+x >= 0) and (j+y >= 0):
                                dil_pix_list.append(pixel[i+x,j+y])
                
                max_pix = max(dil_pix_list)
                for y in range(-2,3,1):
                    for x in range(-2,3,1):
                        if kernel[y+2,x+2] == 1:
                            if (i+x < coulmn) and (j+y < row) and (i+x >= 0) and (j+y >= 0):
                                img_new.putpixel((i+x,j+y),max_pix)
    return img_new

def erosion(img, kernel):
    pixel = img.load()
    coulmn,row=img.size
    img_new = Image.new(img.mode, img.size )
    for i in range(0,coulmn,1):
        for j in range(0,row,1):
            ero_flag = True
            ero_pix_list = []
            for y in range(-2,3,1):
                for x in range(-2,3,1):
                    if kernel[y+2,x+2] == 1:
                        if (i+x < coulmn) and (j+y < row) and (i+x >= 0) and (j+y >= 0):
                            ero_pix_list.append(pixel[i+x,j+y])
                            if pixel[i+x,j+y] == 0:
                                ero_flag = False
                        else:
                            ero_flag = False
            
            min_pix = min(ero_pix_list)
            if ero_flag :
                img_new.putpixel((i,j),min_pix)
                
    return img_new

def opening(img, kernel):
    img_ero = erosion(img, kernel)
    img_new = dilation(img_ero, kernel)

    return img_new

def closing(img, kernel):
    img_dil = dilation(img, kernel)
    img_new = erosion(img_dil, kernel)

    return img_new

def SNR_calculate(img_orig, img_proc):
    pixel_orig = img_orig.load()
    pixel_proc = img_proc.load()
    
    orig_array = np.array((512,512))
    proc_array = np.array((512,512))
    mu = 0
    mu_n = 0
    VS = 0
    VN = 0
    for i in range(0,512,1):
        for j in range(0,512,1):
            mu += pixel_orig[i,j]
            mu_n += pixel_proc[i,j] - pixel_orig[i,j]

    mu = mu / (512*512)
    mu_n = mu_n / (512*512)

    for i in range(0,512,1):
        for j in range(0,512,1):
            VS += (pixel_orig[i,j] - mu) ** 2
            VN += (pixel_proc[i,j] - pixel_orig[i,j] - mu_n) ** 2

    VS = VS / (512*512)
    VN = VN / (512*512)

    SNR = 20 * np.log10( np.sqrt(VS) / np.sqrt(VN) )
    return SNR

def image_processing(img, file_name):
    img_box_3 , img_median_3 = box_and_median_filter(img, 3)
    img_box_5 , img_median_5 = box_and_median_filter(img, 5)

    kernel_array = np.array([[0,1,1,1,0],
                             [1,1,1,1,1],
                             [1,1,1,1,1],
                             [1,1,1,1,1],
                             [0,1,1,1,0]])
    img_open = opening(img, kernel_array)
    img_close = closing(img, kernel_array)
    img_close_open = opening(img_close, kernel_array)
    img_open_close = closing(img_open, kernel_array) 

    img_box_3.save('./processed/' + file_name + '_box_3.bmp')
    img_box_5.save('./processed/' + file_name + '_box_5.bmp')
    img_median_3.save('./processed/' + file_name + '_median_3.bmp')
    img_median_5.save('./processed/' + file_name + '_median_5.bmp')
    img_close_open.save('./processed/' + file_name + '_close_open.bmp')
    img_open_close.save('./processed/' + file_name + '_open_close.bmp')

    print ( file_name + "_box_3 : " + str( SNR_calculate(img, img_box_3) ))
    print ( file_name + "_box_5 : " + str( SNR_calculate(img, img_box_5) ))
    print ( file_name + "_median_3 : " + str( SNR_calculate(img, img_median_3) ))
    print ( file_name + "_median_5 : " + str( SNR_calculate(img, img_median_5) ))
    print ( file_name + "_close_open : " + str( SNR_calculate(img, img_close_open) ))
    print ( file_name + "_open_close : " + str( SNR_calculate(img, img_open_close) ))


lena=Image.open("lena.bmp")

img_noise = Gaussian_noise(lena, 10)
img_noise.save('./noised/Gaussian_10.bmp')
image_processing(img_noise, 'GS_10')

img_noise = Gaussian_noise(lena, 30)
img_noise.save('./noised/Gaussian_30.bmp')
image_processing(img_noise, 'GS_30')

img_noise = salt_and_pepper_noise(lena, 0.05)
img_noise.save('./noised/Salt_and_Pepper_005.bmp')
image_processing(img_noise, 'S&P_005')

img_noise = salt_and_pepper_noise(lena, 0.1)
img_noise.save('./noised/Salt_and_Pepper_01.bmp')
image_processing(img_noise, 'S&P_01')

GS_10_box_3 : 13.132161646862954
GS_10_box_5 : 11.572168146895429
GS_10_median_3 : 13.27631615310952
GS_10_median_5 : 12.100599897080322
GS_10_close_open : 3.6945950492001565
GS_10_open_close : 4.136362133470649
GS_30_box_3 : 5.883109108752843
GS_30_box_5 : 5.358117227626292
GS_30_median_3 : 5.656054991366601
GS_30_median_5 : 5.330359200107821
GS_30_close_open : 2.14640323133101
GS_30_open_close : 3.012827389645247
S&P_005_box_3 : 3.5090408586412805
S&P_005_box_5 : 3.092290603894414
S&P_005_median_3 : 3.051813938604048
S&P_005_median_5 : 2.9498254216201203
S&P_005_close_open : 0.7598616933033864
S&P_005_open_close : 1.1032935636094314
S&P_01_box_3 : 1.9505945538037295
S&P_01_box_5 : 1.592999807303005
S&P_01_median_3 : 1.4311950233094124
S&P_01_median_5 : 1.3615386678530694
S&P_01_close_open : -1.926262016939736
S&P_01_open_close : -1.0983081749830665


In [89]:
from PIL import Image, ImageDraw
import numpy as np

def Gaussian_noise(img, amp):
    pixel = img.load()
    Gaussian_noise_img = Image.new(img.mode, img.size)
    
    for i in range(0,512,1):
        for j in range(0,512,1):
            Gaussian_noise_img.putpixel((i,j),int( pixel[i,j] + amp * np.random.normal(0,1)) ) 
            
    return Gaussian_noise_img

def salt_and_pepper_noise(img, threshold):
    pixel = img.load()
    salt_and_pepper_noise_img = Image.new(img.mode, img.size)
    
    for i in range(0,512,1):
        for j in range(0,512,1):
            rand = np.random.sample()
            if rand < threshold:
                salt_and_pepper_noise_img.putpixel((i,j),0)
            elif rand > 1 - threshold:
                salt_and_pepper_noise_img.putpixel((i,j),255)
            else:
                salt_and_pepper_noise_img.putpixel((i,j),pixel[i,j])
    return salt_and_pepper_noise_img

def box_and_median_filter(img, box_size):
    pixel = img.load()
    img_box = Image.new(img.mode, img.size)
    img_median = Image.new(img.mode, img.size)
    
    x_start = int(box_size/2)
    y_start = int(box_size/2)
    for i in range(0,512,1):
        for j in range(0,512,1):
            box_collect = []
            for x in range(0,box_size,1):
                for y in range(0,box_size,1):
                    try:
                        box_collect.append( pixel[i+x-x_start, j+y-y_start] )
                    except:
                        pass
            img_box.putpixel((i,j) , int(np.mean(np.array(box_collect))) )
            img_median.putpixel((i,j) ,int( np.median(np.array(box_collect))) )
    return img_box, img_median

def dilation(binary,kernal,coulmn,row):
    pix=binary.load()
    img_new=Image.new(binary.mode, binary.size)
    for i in range(coulmn):
        for j in range(row):
            if pix[i,j] > 0:
                pixel_list=[]
                for x in range(-2,3,1):
                    for y in range(-2,3,1):
                        if kernal[x+2][y+2] == 1:
                            if (i+y >=0) and (j+x >= 0) and (i+y < coulmn) and (j+x < row):
                                pixel_list.append(pix[i+y,j+x])
                                
                Max = max(pixel_list)
                for x in range(-2,3,1):
                    for y in range(-2,3,1):
                        if kernal[x+2][y+2] == 1:
                            if (i+y >=0) and (j+x >= 0) and (i+y < coulmn) and (j+x < row):
                                img_new.putpixel((i,j),Max)
    return img_new


def erosion(binary,kernal,coulmn,row):
    pix=binary.load()
    img_new=Image.new(binary.mode, binary.size)
    for i in range(coulmn):
        for j in range(row):
            draw=1
            pixel_list=[]
            for x in range(-2,3,1):
                for y in range(-2,3,1):
                    if kernal[x+2][y+2] == 1:
                        if (i+y < coulmn) and (j+x < row) and (i+y >= 0) and (j+x >= 0):
                            pixel_list.append(pix[i+y,j+x])
                            if pix[i+y,j+x]==0:
                                draw=0
                        else:
                            draw=0
            Min = min(pixel_list)
            if draw == 1:
                img_new.putpixel((i,j),Min)
    return img_new


def opening(binary,kernal,coulmn,row):
    p=erosion(binary,kernal,coulmn,row)
    op=dilation(p,kernal,coulmn,row)

    return op

def closing(binary,kernal,coulmn,row):
    p=dilation(binary,kernal,coulmn,row)
    close=erosion(p,kernal,coulmn,row)

    return close

def SNR_calculate(img, img_pro):
    pixel = img.load()
    pixel_pro = img_pro.load()
    
    orig_array = np.array((512,512))
    proc_array = np.array((512,512))
    mu = 0
    mu_n = 0
    VS = 0
    VN = 0
    for i in range(0,512,1):
        for j in range(0,512,1):
            mu += pixel[i,j]
            mu_n += pixel_pro[i,j] - pixel[i,j]

    mu = mu / (512*512)
    mu_n = mu_n / (512*512)

    for i in range(0,512,1):
        for j in range(0,512,1):
            VS += (pixel[i,j] - mu) ** 2
            VN += (pixel_pro[i,j] - pixel[i,j] - mu_n) ** 2

    VS = VS / (512*512)
    VN = VN / (512*512)

    SNR = 20 * np.log10( np.sqrt(VS) / np.sqrt(VN) )
    return SNR

def image_processing(img, file_name,coulmn,row):
    img_box_3 , img_median_3 = box_and_median_filter(img, 3)
    img_box_5 , img_median_5 = box_and_median_filter(img, 5)

    kernel_array = np.array([[0,1,1,1,0],
                             [1,1,1,1,1],
                             [1,1,1,1,1],
                             [1,1,1,1,1],
                             [0,1,1,1,0]])
    img_open = opening(img, kernel_array,coulmn,row)
    img_close = closing(img, kernel_array,coulmn,row)
    img_close_open = opening(img_close, kernel_array,coulmn,row)
    img_open_close = closing(img_open, kernel_array,coulmn,row) 

    img_box_3.save('./processed/' + file_name + '_box_3.bmp')
    img_box_5.save('./processed/' + file_name + '_box_5.bmp')
    img_median_3.save('./processed/' + file_name + '_median_3.bmp')
    img_median_5.save('./processed/' + file_name + '_median_5.bmp')
    img_close_open.save('./processed/' + file_name + '_close_open.bmp')
    img_open_close.save('./processed/' + file_name + '_open_close.bmp')

    print ( file_name + "_box_3 : " + str( SNR_calculate(img, img_box_3) ))
    print ( file_name + "_box_5 : " + str( SNR_calculate(img, img_box_5) ))
    print ( file_name + "_median_3 : " + str( SNR_calculate(img, img_median_3) ))
    print ( file_name + "_median_5 : " + str( SNR_calculate(img, img_median_5) ))
    print ( file_name + "_close_open : " + str( SNR_calculate(img, img_close_open) ))
    print ( file_name + "_open_close : " + str( SNR_calculate(img, img_open_close) ))


lena=Image.open("lena.bmp")
pix=lena.load()
coulmn,row=lena.size

img_noise = Gaussian_noise(lena, 10)
img_noise.save('./noised/Gaussian_10.bmp')
image_processing(img_noise, 'GS_10',coulmn,row)

img_noise = Gaussian_noise(lena, 30)
img_noise.save('./noised/Gaussian_30.bmp')
image_processing(img_noise, 'GS_30',coulmn,row)\

mg_noise = salt_and_pepper_noise(lena, 0.1)
img_noise.save('./noised/Salt_and_Pepper_01.bmp')
image_processing(img_noise, 'S&P_01',coulmn,row)

img_noise = salt_and_pepper_noise(lena, 0.05)
img_noise.save('./noised/Salt_and_Pepper_005.bmp')
image_processing(img_noise, 'S&P_005',coulmn,row)


GS_10_box_3 : 13.122805645603712
GS_10_box_5 : 11.55651452734713
GS_10_median_3 : 13.255413007523718
GS_10_median_5 : 12.07642678099068
GS_10_close_open : 4.9853651252219375
GS_10_open_close : 6.007313960926206
GS_30_box_3 : 5.908953190216726
GS_30_box_5 : 5.381307572919014
GS_30_median_3 : 5.698072156022352
GS_30_median_5 : 5.353195084791844
GS_30_close_open : -0.4663276891145245
GS_30_open_close : 3.0745594385376456
S&P_005_box_3 : 3.4376434457973355
S&P_005_box_5 : 3.024018757319112
S&P_005_median_3 : 2.9746811762923517
S&P_005_median_5 : 2.8684658281065225
S&P_005_close_open : -0.4377127603078939
S&P_005_open_close : -0.33241676659960895
S&P_01_box_3 : 1.9725161796598254
S&P_01_box_5 : 1.6134476367356452
S&P_01_median_3 : 1.4558407169169856
S&P_01_median_5 : 1.3868425936872213
S&P_01_close_open : -0.013651626505885334
S&P_01_open_close : -0.004946394691192808


In [87]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import sys

def Gaussian_noise(img_orig, amp):
    pixel_orig = img_orig.load()
    img_ret = Image.new(img_orig.mode, img_orig.size)
    pixel_ret = img_ret.load()
    
    for i in range(0,512,1):
        for j in range(0,512,1):
            pixel_ret[i,j] = int( pixel_orig[i,j] + amp * np.random.normal(0,1) )
    
    return img_ret

def salt_and_pepper_noise(img_orig, thres):
    pixel_orig = img_orig.load()
    img_ret = Image.new(img_orig.mode, img_orig.size)
    pixel_ret = img_ret.load()
    
    for i in range(0,512,1):
        for j in range(0,512,1):
            rand = np.random.sample()
            if rand < thres:
                pixel_ret[i,j] = 0
            elif rand > 1 - thres:
                pixel_ret[i,j] = 255
            else:
                pixel_ret[i,j] = pixel_orig[i,j]  
    
    return img_ret

def box_and_median_filter(img_orig, box_size):
    pixel_orig = img_orig.load()
    img_box = Image.new(img_orig.mode, img_orig.size)
    pixel_box = img_box.load()
    img_median = Image.new(img_orig.mode, img_orig.size)
    pixel_median = img_median.load()
    
    x_start = int(box_size/2)
    y_start = int(box_size/2)
    for i in range(0,512,1):
        for j in range(0,512,1):
            box_collect = []
            for x in range(0,box_size,1):
                for y in range(0,box_size,1):
                    try:
                        box_collect.append( pixel_orig[i+x-x_start, j+y-y_start] )
                    except:
                        pass
            # print(np.mean(np.array(box_collect)))
            pixel_box[i,j] = int( np.mean(np.array(box_collect)) )
            pixel_median[i,j] = int( np.median(np.array(box_collect)) )

    return img_box, img_median

def dilation(img, kernel):
    pixel = img.load()
    img_ret = Image.new(img.mode, img.size )
    pixel_ret = img_ret.load()
    for i in range(0,img.width,1):
        for j in range(0,img.height,1):
            if pixel[i,j] > 0:
                dil_pix_list = []
                for y in range(-2,kernel.shape[1]-2,1):
                    for x in range(-2,kernel.shape[0]-2,1):
                        if kernel[y+2,x+2] == 1:
                            if (i+x < img.width) and (j+y < img.height) and (i+x >= 0) and (j+y >= 0):
                                dil_pix_list.append(pixel[i+x,j+y])
                
                max_pix = max(dil_pix_list)
                for y in range(-2,kernel.shape[1]-2,1):
                    for x in range(-2,kernel.shape[0]-2,1):
                        if kernel[y+2,x+2] == 1:
                            if (i+x < img.width) and (j+y < img.height) and (i+x >= 0) and (j+y >= 0):
                                pixel_ret[i+x,j+y] = max_pix
    return img_ret

def erosion(img, kernel):
    pixel = img.load()
    img_ret = Image.new(img.mode, img.size )
    pixel_ret = img_ret.load()
    for i in range(0,img.width,1):
        for j in range(0,img.height,1):
            ero_flag = True
            ero_pix_list = []
            for y in range(-2,kernel.shape[1]-2,1):
                for x in range(-2,kernel.shape[0]-2,1):
                    if kernel[y+2,x+2] == 1:
                        if (i+x < img.width) and (j+y < img.height) and (i+x >= 0) and (j+y >= 0):
                            ero_pix_list.append(pixel[i+x,j+y])
                            if pixel[i+x,j+y] == 0:
                                ero_flag = False
                        else:
                            ero_flag = False
            
            min_pix = min(ero_pix_list)
            if ero_flag :
                pixel_ret[i,j] = min_pix
                
    return img_ret

def opening(img, kernel):
    img_ero = erosion(img, kernel)
    img_ret = dilation(img_ero, kernel)

    return img_ret

def closing(img, kernel):
    img_dil = dilation(img, kernel)
    img_ret = erosion(img_dil, kernel)

    return img_ret

def SNR_calculate(img_orig, img_proc):
    pixel_orig = img_orig.load()
    pixel_proc = img_proc.load()
    
    orig_array = np.array((512,512))
    proc_array = np.array((512,512))
    mu = 0
    mu_n = 0
    VS = 0
    VN = 0
    for i in range(0,512,1):
        for j in range(0,512,1):
            mu += pixel_orig[i,j]
            mu_n += pixel_proc[i,j] - pixel_orig[i,j]

    mu = mu / (512*512)
    mu_n = mu_n / (512*512)

    for i in range(0,512,1):
        for j in range(0,512,1):
            VS += (pixel_orig[i,j] - mu) ** 2
            VN += (pixel_proc[i,j] - pixel_orig[i,j] - mu_n) ** 2

    VS = VS / (512*512)
    VN = VN / (512*512)

    SNR = 20 * np.log10( np.sqrt(VS) / np.sqrt(VN) )
    return SNR

def image_processing(img, file_name):
    img_box_3 , img_median_3 = box_and_median_filter(img, 3)
    img_box_5 , img_median_5 = box_and_median_filter(img, 5)

    kernel_array = np.array([[0,1,1,1,0],
                             [1,1,1,1,1],
                             [1,1,1,1,1],
                             [1,1,1,1,1],
                             [0,1,1,1,0]])
    img_open = opening(img, kernel_array)
    img_close = closing(img, kernel_array)
    img_close_open = opening(img_close, kernel_array)
    img_open_close = closing(img_open, kernel_array) 

    img_box_3.save('./processed/' + file_name + '_box_3.bmp')
    img_box_5.save('./processed/' + file_name + '_box_5.bmp')
    img_median_3.save('./processed/' + file_name + '_median_3.bmp')
    img_median_5.save('./processed/' + file_name + '_median_5.bmp')
    img_close_open.save('./processed/' + file_name + '_close_open.bmp')
    img_open_close.save('./processed/' + file_name + '_open_close.bmp')

    print ( file_name + "_box_3 : " + str( SNR_calculate(img, img_box_3) ))
    print ( file_name + "_box_5 : " + str( SNR_calculate(img, img_box_5) ))
    print ( file_name + "_median_3 : " + str( SNR_calculate(img, img_median_3) ))
    print ( file_name + "_median_5 : " + str( SNR_calculate(img, img_median_5) ))
    print ( file_name + "_close_open : " + str( SNR_calculate(img, img_close_open) ))
    print ( file_name + "_open_close : " + str( SNR_calculate(img, img_open_close) ))

def main(argv):
    img_input = Image.open(argv)
    pixel_input = img_input.load()
    
    img_noise = Gaussian_noise(img_input, 10)
    img_noise.save('./noised/Gaussian_10.bmp')
    image_processing(img_noise, 'GS_10')

    img_noise = Gaussian_noise(img_input, 30)
    img_noise.save('./noised/Gaussian_30.bmp')
    image_processing(img_noise, 'GS_30')

    img_noise = salt_and_pepper_noise(img_input, 0.05)
    img_noise.save('./noised/Salt_and_Pepper_005.bmp')
    image_processing(img_noise, 'S&P_005')
    
    img_noise = salt_and_pepper_noise(img_input, 0.1)
    img_noise.save('./noised/Salt_and_Pepper_01.bmp')
    image_processing(img_noise, 'S&P_01')
main('lena.bmp')

GS_10_box_3 : 13.133808993525273
GS_10_box_5 : 11.562821398718192
GS_10_median_3 : 13.283899582273655
GS_10_median_5 : 12.097215465899255
GS_10_close_open : 3.682970144946763
GS_10_open_close : 4.1374795391921975
GS_30_box_3 : 5.883047165455894
GS_30_box_5 : 5.348223728665116
GS_30_median_3 : 5.669843228313589
GS_30_median_5 : 5.322369586884665
GS_30_close_open : 2.1248667481282206
GS_30_open_close : 3.000194582445663
S&P_005_box_3 : 3.5132262803466596
S&P_005_box_5 : 3.0998458856079516
S&P_005_median_3 : 3.050671781009717
S&P_005_median_5 : 2.9527193011143007
S&P_005_close_open : 0.8698401961404008
S&P_005_open_close : 1.1081894618312256
S&P_01_box_3 : 1.9809517760062048
S&P_01_box_5 : 1.6097590643269923
S&P_01_median_3 : 1.456794685197773
S&P_01_median_5 : 1.380799482626673
S&P_01_close_open : -1.8531880915920058
S&P_01_open_close : -1.0642069090494013
